In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

- Match depth images to the RGB images
- Read the first image from file and it's ground truth pose as reference.
- After the first image, read one image at a time and calculate it's tranformation from the previous image.
    - To calculate the tranformation, first extract SIFT descriptors and for both images and find matches. Then use the SVD based formula to calculate the transformation from the images and apply the tranformation to the pose of previous time instant to calculate the new pose. Convert the rotation matrix into unit quaternions.
    

In [2]:
with open('matched.txt', 'r') as f:
    data = f.read()
lines = data.replace(","," ").replace("\t"," ").split("\n") 
matches = [[v.strip() for v in line.split(" ") if v.strip()!=""] for line in lines if len(line)>0 and line[0]!="#"]


In [3]:
def visualize_matches(img1, img2, src_pts, dst_pts, kp1, kp2, good):
    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
    matchesMask = mask.ravel().tolist()
    h,w = img1.shape
    pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
    dst = cv2.perspectiveTransform(pts,M)
    img2 = cv2.polylines(img2,[np.int32(dst)],True,255,3, cv2.LINE_AA)

    draw_params = dict(matchColor = (0,255,0), # draw matches in green color
                   singlePointColor = None,
                   matchesMask = matchesMask, # draw only inliers
                   flags = 2)

    img3 = cv2.drawMatches(img1,kp1,img2,kp2,good,None,**draw_params)
    
    plt.imshow(img3, 'gray'),plt.show()

In [4]:
def compute_matches(img1, img2):
    sift = cv2.xfeatures2d.SIFT_create()
    MIN_MATCH_COUNT = 10

    # find the keypoints and descriptors with SIFT
    kp1, des1 = sift.detectAndCompute(img1,None)
    kp2, des2 = sift.detectAndCompute(img2,None)

    FLANN_INDEX_KDTREE = 0
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks = 50)

    flann = cv2.FlannBasedMatcher(index_params, search_params)

    matches = flann.knnMatch(des1,des2,k=2)

    # store all the good matches as per Lowe's ratio test.
    good = []
    for m,n in matches:
        if m.distance < 0.7*n.distance:
            good.append(m)
            
    if len(good)>MIN_MATCH_COUNT:
        src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
        dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)
#         visualize_matches(img1, img2, src_pts, dst_pts, kp1, kp2, good)
        return src_pts, dst_pts
    else:
        print("Not enough matches are found")
        matchesMask = None

In [5]:
def transform_points(pts_1, pts_2, depths_1, depths_2):
    depths_1 = np.array([depths_1]).T
    depths_2 = np.array([depths_2]).T
    
    pts_1 = np.append(pts_1, depths_1, axis=1)
    pts_2 = np.append(pts_2, depths_2, axis=1)
    all_points = np.append(pts_1, pts_2, axis=1)
    all_points = all_points[np.where(np.logical_and(all_points[:,2] > 0, all_points[:,5] > 0))]

    pts_1 = all_points[:,:3]
    pts_1[:,0] = (pts_1[:,0] - 239.5)/525.0
    pts_1[:,0] = pts_1[:,0]*pts_1[:,2]
    pts_1[:,1] = (pts_1[:,1] - 319.5)/525.0
    pts_1[:,1] = pts_1[:,1]*pts_1[:,2]

    pts_2 = all_points[:,3:]
    pts_2[:,0] = (pts_2[:,0] - 239.5)/525.0
    pts_2[:,0] = pts_2[:,0]*pts_2[:,2]
    pts_2[:,1] = (pts_2[:,1] - 319.5)/525.0
    pts_2[:,1] = pts_2[:,1]*pts_2[:,2]
    return pts_1, pts_2

In [6]:
def calculate_transformation(pts_1, pts_2):
    """
    Given two sets of points calculate the rotation and translation of the camera pose
    """
    mean_1 = np.mean(pts_1, axis=0)
    mean_2 = np.mean(pts_2, axis=0)

    pts_1 = pts_1 - mean_1
    pts_2 = pts_2 - mean_2

    mat = np.dot(pts_1.T, pts_2)
    u, s, v = np.linalg.svd(mat)
    R = np.dot(v, u.T)
    t = mean_2 - np.dot(R,mean_1)
    
    return R, t

In [7]:
def get_quaternions(R):
    tr = R[0][0] + R[1][1] + R[2][2]

    if (tr > 0):
        S = np.sqrt(tr+1.0) * 2
        qw = 0.25 * S;
        qx = (R[2][1] - R[1][2]) / S
        qy = (R[0][2] - R[2][0]) / S 
        qz = (R[1][0] - R[0][1]) / S
    elif((R[0][0] > R[1][1]) and (R[0][0] > R[2][2])):
        S = np.sqrt(1.0 + R[0][0] - R[1][1] - R[2][2]) * 2
        qw = (R[2][1] - R[1][2]) / S
        qx = 0.25 * S
        qy = (R[0][1] + R[1][0]) / S 
        qz = (R[0][2] + R[2][0]) / S
    elif (R[1][1] > R[2][2]): 
        S = np.sqrt(1.0 + R[1][1] - R[0][0] - R[2][2]) * 2
        qw = (R[0][2] - R[2][0]) / S
        qx = (R[0][1] + R[1][0]) / S 
        qy = 0.25 * S
        qz = (R[1][2] + R[2][1]) / S
    else:
        S = np.sqrt(1.0 + R[2][2] - R[0][0] - R[1][1]) * 2
        qw = (R[1][0] - R[0][1]) / S
        qx = (R[0][2] + R[2][0]) / S
        qy = (R[1][2] + R[2][1]) / S
        qz = 0.25 * S;
    return qx, qy, qz, qw

In [8]:
def get_pose_matrix(pose):
    pose_matrix = np.zeros((4,4))
    t = pose[0]
    q = pose[1]
    x = q[0]
    y = q[1]
    z = q[2]
    w = q[3]
    R = np.zeros((3,3))
    R[0][0] = 1 - 2*(y**2 + z**2)
    R[0][1] = 2*(x*y - z*w)
    R[0][2] = 2*(x*z + y*w)
    R[1][0] = 2*(x*y + z*w)
    R[1][1] = 1 - 2*(x**2 + z**2)
    R[1][2] = 2*(y*z - x*w)
    R[2][0] = 2*(x*z - y*w)
    R[2][1] = 2*(y*z + x*w)
    R[2][2] = 1 - 2*(x**2 + y**2)
    
    pose_matrix[:3,:3] = R
    pose_matrix[:3, 3] = t
    pose_matrix[3,3] = 1
    return pose_matrix

In [9]:
index = 0
pose_1 = [np.array([1.3434, 0.6271, 1.6606]), np.array([0.6583, 0.6112, -0.2938, -0.3266])]
rgbimage_1 = cv2.imread(matches[index][3], 0)
depthimage_1 = cv2.imread(matches[index][1], -1)
index += 1
while(index < len(matchess)):
    new_rgbimage = cv2.imread(matches[index][3], 0)
    new_depthimage = cv2.imread(matches[index][1], -1)
    src_pts, dst_pts = compute_matches(new_rgbimage, rgbimage_1)
    
    pts_1 = np.squeeze(src_pts)
    pts_2 = np.squeeze(dst_pts)
    
    depths_1 = depthimage_1[np.int32(pts_1[:, 1]), np.int32(pts_1[:, 0])]/5000
    depths_2 = new_depthimage[np.int32(pts_2[:, 1]), np.int32(pts_2[:, 0])]/5000
    
    pts_1, pts_2 = transform_points(pts_1, pts_2, depths_1, depths_2)
    
    R, t = calculate_transformation(pts_1, pts_2)
    new_posematrix = np.zeros((4,4))
    new_posematrix[:3,:3] = R
    new_posematrix[:3,3] = t
    new_posematrix[3,3] = 1
    
    posematrix_1 = get_pose_matrix(pose_1)
    result_pose = np.dot(posematrix_1, new_posematrix)
    
    qx, qy, qz, qw = get_quaternions(result_pose[:3,:3])
    pose_1 = [new_posematrix[:3,3], np.array([qx, qy, qz, qw])]
#     qx, qy, qz, qw = get_quaternions(R)
#     v1 = np.array([qx, qy, qz])
#     w1 = qw
#     v0 = pose_1[1][:3]
#     w0 = pose_1[1][3]
# #     q 2 = q 0 q 1 = (v 0 × v 1 + w 0 v 1 + w 1 v 0 , w 0 w 1 − v 0 · v 1 )
#     new_v = np.cross(v0,v1) + w0*v1 + w1*v0
#     new_w = w0*w1 - np.dot(v0,v1)
    
#     pose_1 = [t, np.append(new_v, new_w)]
    rgbimage_1 = new_rgbimage
    depthimage_1 = new_depthimage
    print(matches[index][0], pose_1)
    index += 1

1305031102.226738 [array([-0.14356612, -1.07928301,  0.4403204 ]), array([-0.1931933 ,  0.9763566 ,  0.09655318,  0.00123194])]
1305031102.262886 [array([ 0.18049132, -0.99793439,  0.31389873]), array([ 0.78456352, -0.42887962, -0.31108499, -0.32196764])]
1305031102.295279 [array([ 0.24839762, -0.91368663,  0.30715792]), array([ 0.74832467,  0.44045024, -0.29226428, -0.40064757])]
1305031102.329195 [array([ 0.26527623, -0.9290569 ,  0.30779214]), array([ 0.14536328,  0.97137422, -0.05023226, -0.180868  ])]
1305031102.363013 [array([ 0.16940986, -0.99605179,  0.37776175]), array([-0.5957236 ,  0.73494825,  0.25364856,  0.20139412])]
1305031102.394772 [array([-0.04102452, -1.13731338,  0.49987088]), array([ 0.81152471,  0.24194956, -0.35691489, -0.39425175])]
1305031102.427815 [array([ 0.0624689 , -0.95193271,  0.36992353]), array([ 0.23303094,  0.94218133, -0.11195425, -0.21307703])]
1305031102.462395 [array([-0.056387  , -1.16320625,  0.44831283]), array([-0.5930741 ,  0.73811831,  0.2

In [ ]:
#1305031102.160407 1.344379 0.627206 1.661754 0.658249 0.611043 -0.294444 -0.326553
#1305031102.194330 1.343641 0.626458 1.652408 0.657327 0.613265 -0.295150 -0.323593
#1305031102.226738 1.338382 0.625665 1.641460 0.657713 0.615255 -0.294626 -0.319485
#1305031102.262886 1.325627 0.624485 1.632561 0.659141 0.617445 -0.292536 -0.314195